In [1]:
import numpy as np
import pandas as pd
df=pd.read_csv('Telco-Customer-Churn.csv')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [5]:
df=df.drop('customerID',axis='columns')

In [6]:
df['TotalCharges']=df['TotalCharges'].replace(' ',None)
df['TotalCharges'].isnull().unique()
df['TotalCharges'] = df['TotalCharges'].astype('float')

In [7]:
cat=[i for i in df.select_dtypes(include=['object']).columns]

In [8]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

for i in cat:
    df[i]= label_encoder.fit_transform(df[i])

In [9]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


In [10]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2,random_state=10)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

res={}
estimators = [KNeighborsClassifier(),LogisticRegression(),GaussianNB(),DecisionTreeClassifier(),SVC(),RandomForestClassifier()]

from sklearn.metrics import confusion_matrix,accuracy_score

In [12]:
def bag():
    b = np.random.choice(range(len(y_train)),int(len(y_train)/len(estimators)),replace=False)
    return x_train[b],y_train[b]

In [13]:
for e in estimators:
    xt,yt=bag()
    e.fit(xt,yt)
    res[str(e)]=accuracy_score(y_test,e.predict(x_test))

In [14]:
print(res)

{'KNeighborsClassifier()': 0.7650816181689141, 'LogisticRegression()': 0.8097941802696949, 'GaussianNB()': 0.7395315826827538, 'DecisionTreeClassifier()': 0.7260468417317246, 'SVC()': 0.7565649396735273, 'RandomForestClassifier()': 0.8005677785663591}


In [15]:
r=[]
for mod in estimators:
    y_pred = mod.predict(x_test)
    r.append(y_pred)
r=np.array(r)
from statistics import mode
en_res=np.array([mode(i) for i in r.T])
print("Accuracy of the model: ",accuracy_score(y_test,en_res))

Accuracy of the model:  0.8076650106458482
